# Step 3B: BERT Model 2 - BERT sentiment analysis using US airline sentiment dataset (without equalizing sentiments)

# Without normalizing label count in twitter us airline sentiment dataset

## 1. Required imports

### 1.1 Import required libraries

In [1]:
#---Neural Network libraries
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup            #huggingface transformers library
import tensorflow as tf                                                                                                  #tensorflow library         
import torch                                                                                                             #pytorch library
import torch.nn.functional as Func
import gc
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler, SequentialSampler                        #for loading data into our model

#---Data processing
from sklearn.model_selection import train_test_split                                                                     #for splitting data into training, testing and validation
from sklearn.metrics import f1_score
import pandas as pd                                                                                                      #for using data in the form of dataframes
import numpy as np
import re                                                                                                                #for data manipulation when cleaning datasets

#---Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# from transformers import InputExample, InputFeatures

In [2]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

### 1.2 Import training data to train the BERT model
- *This dataset is from kaggle - https://www.kaggle.com/crowdflower/twitter-airline-sentiment?select=Tweets.csv*
- *We only pick the required columns from the dataset*

In [12]:
training_data = pd.read_csv(r"C:\Users\Karthik\Desktop\Dissertation\BERT\Training dataset\Tweets.csv")
training_data = training_data[['tweet_id', 'airline_sentiment', 'text']]
training_data.set_index('tweet_id', inplace=True)
training_data.head()

,airline_sentiment,text
tweet_id,,
570306133677760513,neutral,@VirginAmerica What @dhepburn said.
570301130888122368,positive,@VirginAmerica plus you've added commercials t...
570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...
570301031407624196,negative,@VirginAmerica it's really aggressive to blast...
570300817074462722,negative,@VirginAmerica and it's a really big bad thing...


### 1.3 Import reddit and twitter data for which we have to populate the sentiments
- *we will use them on the model after the model is trained with the training dataset from kaggle*
- *we save data in two dictionaries; one for reddit and one for twitter to make it more organized*

In [3]:
tick_list =  ['AAPL', 'AMC', 'DKNG', 'TSLA', 'AMD', 'BABA']

In [4]:
reddit_df_dict = {tick: pd.read_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Reddit\\consolidated_pickle_files\\reddit_{tick}_df_for_BERT.pkl") for tick in tick_list}
# reddit_df_dict  

In [5]:
reddit_df_dict['TSLA']

,body,created_utc,id,top,year,month,day,ticker,YearMonDay
0,Non-troll post. I started 2 weeks ago and have...,1609500199,ghp72zs,top,21,Jan,01,TSLA,21Jan01
1,we eat cornbread on new years day to ensure a ...,1609524808,ghqo6qb,top,21,Jan,01,TSLA,21Jan01
2,TSLA 850 EOD,1612203941,glmp0gv,top,21,Feb,01,TSLA,21Feb01
3,joe weisenthal is the ultimate chad( who else ...,1612207510,glmyg66,top,21,Feb,01,TSLA,21Feb01
4,TSLA 🚀🚀🚀🚀🚀,1612211972,gln9v98,top,21,Feb,01,TSLA,21Feb01
...,...,...,...,...,...,...,...,...,...
15125,TSLA Drill Team 6 reporting for duty,1609359381,ghj65jr,top,20,Dec,30,TSLA,20Dec30
15126,TSLA’s still having them TSLA days I see. Join...,1609359393,ghj66f1,top,20,Dec,30,TSLA,20Dec30
15127,TSLA 🎰 EOY coming,1609360555,ghj8jam,top,20,Dec,30,TSLA,20Dec30
15128,Thank fuck I didn't dump my TSLA calls yesterday,1609361591,ghjamup,top,20,Dec,30,TSLA,20Dec30


In [6]:
twitter_df_dict = {tick: pd.read_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Twitter\\consolidated_pickle_files\\twitter_{tick}_df_for_BERT.pkl") for tick in tick_list}

In [7]:
twitter_df_dict['TSLA']

,date,content,likeCount,lang,cleaned_content,year,month,day,ticker,YearMonDay
0,2021-09-18 19:56:59+00:00,@The_RockTrading Bullish on $TSLA this week &a...,1,en,Bullish on $TSLA this week & $Aapl,21,Sep,18,TSLA,21Sep18
1,2021-09-18 04:16:52+00:00,$TSLA now is the same as $aapl in the 80s ! @e...,1,en,$TSLA now is the same as $aapl in the 80s !,21,Sep,18,TSLA,21Sep18
2,2021-09-16 19:01:28+00:00,"Added more $TSLA and $aapl to long, because I ...",0,en,"Added more $TSLA and $aapl to long, because I ...",21,Sep,16,TSLA,21Sep16
3,2021-09-16 15:23:53+00:00,@NeilRog49855230 @Gays4Tesla @TheMaverickWS Th...,3,en,There is plenty of information available on-li...,21,Sep,16,TSLA,21Sep16
4,2021-09-16 15:23:11+00:00,There is plenty of information available on-li...,0,en,There is plenty of information available on-li...,21,Sep,16,TSLA,21Sep16
...,...,...,...,...,...,...,...,...,...,...
179489,2020-06-01 18:42:22+00:00,Normally volatile Tesla $TSLA is the IBD Stock...,1,en,Normally volatile Tesla $TSLA is the IBD Stock...,20,Jun,01,TSLA,20Jun01
179490,2020-06-01 18:42:10+00:00,Normally volatile Tesla $TSLA is the IBD Stock...,1,en,Normally volatile Tesla $TSLA is the IBD Stock...,20,Jun,01,TSLA,20Jun01
179491,2020-06-01 18:41:35+00:00,$TSLA up $51.00 from next suggested buy entry ...,1,en,$TSLA up $51.00 from next suggested buy entry ...,20,Jun,01,TSLA,20Jun01
179492,2020-06-01 18:41:32+00:00,@Desert_Trader81 $TSLA on the move crossing HO...,0,en,$TSLA on the move crossing HOD $885 !!!! 900 w...,20,Jun,01,TSLA,20Jun01


## 2. Prepare training dataset

### 2.1 One-Hot encode the labels

In [5]:
# training_data = pd.read_csv(r"C:\Users\Karthik\Desktop\Dissertation\BERT\Training dataset\Tweets.csv")
# training_data = training_data[['tweet_id', 'airline_sentiment', 'text']]
# training_data.set_index('tweet_id', inplace=True)
# training_data.head()

In [6]:
training_data.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [7]:
# training_data = pd.concat([training_data.query('airline_sentiment=="negative"').sample(n=2363),training_data.query('airline_sentiment=="neutral"').sample(n=2363),training_data.query('airline_sentiment=="positive"')])

*There are 3 lables so we can encode the lables with 0,1 and 2*

In [13]:
training_data['label'] = training_data.airline_sentiment.replace({'neutral':0, 'positive':1, 'negative':2})
training_data.head()

,airline_sentiment,text,label
tweet_id,,,
570306133677760513,neutral,@VirginAmerica What @dhepburn said.,0
570301130888122368,positive,@VirginAmerica plus you've added commercials t...,1
570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...,0
570301031407624196,negative,@VirginAmerica it's really aggressive to blast...,2
570300817074462722,negative,@VirginAmerica and it's a really big bad thing...,2


### 2.2 Data cleaning

#### 2.2.1 Define data cleaning function

In [8]:
def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text) #remove hashtags
    text = re.sub(r'http\S+', '', text)    #remove urls
    text = re.sub(r'&amp;amp', '&', text)  #remove double amps
    text = re.sub(r'\&amp;', '&', text)    #remove single amps
    text = re.sub(r'\s+', ' ', text)       #reduce multiple spaces into a single space
    return text

#### 2.2.2 Clean the data

In [ ]:
#---Make a copy of the original raw text for future reference if required
training_data['original_text'] = training_data['text']
#---Use the above function to clean the tweet data
training_data['text'] = training_data.text.apply(lambda x: text_preprocessing(x))
training_data.head()

### 2.3 Split the cleaned training data into train and validation datasets
- *we do not need a test dataset. This is because we are using the training dataset only to train the BERT model*
- *since we are going to use the BERT model to classify a different corpus (reddit and twitter data), splitting the data into training and validation datasets serves our purpose*
- *this will also help in having a bigger training dataset*

In [23]:
#---we will use scikit-learn's train_test_split to classify records into train, test or validation dataset
x_train, x_val, y_train, y_val =  train_test_split(training_data.index.values,          #x values/ input values 
                                                   training_data.label.values,          #y values/ output values/ lables
                                                   test_size=0.25,                      #percentage of data to be used for test dataset
                                                   random_state=17,                     #shuffles the data prior to splitting
                                                   stratify=training_data.label.values  #stratification helps when the distribution of data is uneven like in our case where,
                                                                                        #we have a lot of negative compared to neutral and positive labels                                                   
)

#---add a new column to training data to classify records into train and val datasets
training_data['dataset'] = 'NA'
training_data.loc[x_train, 'dataset'] = 'train'
training_data.loc[x_val, 'dataset'] = 'val'

training_data.groupby(['airline_sentiment', 'label', 'dataset']).count()

text  original_text
airline_sentiment label dataset                     
negative          2     train    6849           6849
                        val      2329           2329
neutral           0     train    2311           2311
                        val       788            788
positive          1     train    1760           1760
                        val       603            603

### 2.4 Tokenize and encode data

#### 2.4.1 Find the max tweet length in the training data
- *This is required to be input into the tokenizer*
- *BERT expects all its inputs to be of the same length so shorter sentences will be padded to maintain the length*

In [24]:
MAX_LEN=training_data['text'].str.len().max()

#### 2.4.2 Initialize huggingface tokenizer and tokenize and encode train and val datasets

- *In this step we will be tokenizing and encoding the input side of our datasets (i.e) the tweet comments*
- *We will be using batch_encode_plus() method in the tokenizer as our input to the tokenizer will be an array of the tweet column in training_data [training_data[training_data.dataset=='train'].text.values]*
- Ref:*https://huggingface.co/transformers/internal/tokenization_utils.html#pretrainedtokenizerbase*

In [ ]:
#---Initialize bert-base-uncased tokenizer and select the option to convert all text to lowercase
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [25]:
#---Tokenize and encode train dataset
train_enc = tokenizer.batch_encode_plus(training_data[training_data.dataset=='train'].text.values, 
                                           add_special_tokens=True, 
                                           return_attention_mask=True, 
                                           pad_to_max_length=True, 
                                           max_length=MAX_LEN, 
                                           return_tensors='pt'
                                           )

#---Tokenize and encode val dataset
val_enc = tokenizer.batch_encode_plus(training_data[training_data.dataset=='val'].text.values, 
                                         add_special_tokens=True, 
                                         return_attention_mask=True, 
                                         pad_to_max_length=True, 
                                         max_length=MAX_LEN, 
                                         return_tensors='pt'
                                         )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2184: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


### 2.5 Create tensor datasets from the encoded datasets from previous step
- *We must split the input_ids(encoded tokens) and attention masks(says which values a model should work on since we padded tweets shorter than 172) from the previous step*
- *we must create a tensor dataset with both input(encoded tweets from last step) and also outputs (the one-hot encoded lables)*
- *we create a tensor with the outputs so that the TensorDataset method can combine both inputs and outputs*

In [14]:
#---Get the input ids and attention masks from the tokenizer outputs
train_input_ids = train_enc['input_ids']
train_attn_masks = train_enc['attention_mask']
val_input_ids = val_enc['input_ids']
val_attn_masks = val_enc['attention_mask']

#---Create tensors with the output data
train_labels = torch.tensor(training_data[training_data.dataset=='train'].label.values)
val_labels = torch.tensor(training_data[training_data.dataset=='val'].label.values)

#---Create the required tensor datasets which will be used in the dataloader
train_tensor = TensorDataset(train_input_ids, train_attn_masks, train_labels)
val_tensor = TensorDataset(val_input_ids, val_attn_masks, val_labels)

len(train_tensor), len(val_tensor)

(10920, 3720)

### 2.6 Create dataloaders
- *We create two dataloaders - one for training data and one for validation data*
- *We use random sampling*
- *We use a  batch size of 32 in consideration of the low computing power at hand. It is also known that smaller batch sizes converge quicker - this seems like a good choice considering our limited training data*


In [15]:
batch_size = 32

# We Need two different dataloder
train_dloader = DataLoader(train_tensor, sampler=RandomSampler(train_tensor), batch_size=batch_size)
val_dloader = DataLoader(val_tensor, sampler=RandomSampler(val_tensor), batch_size=batch_size)

In [16]:
# for batch in val_dloader:
#     display(batch)
#     display("----=-=-=-=--=-=")
#     for b in batch:
#         display(b)

## 3. BERT model and performance definitions
- *Define BERT model, optimizer, scheduler and performance metrics*
- *Ignore the warning because we are going to train the model before using it to classify reddit and twitter data

### 3.1 Define the BERT model

In [17]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(training_data.label.unique()),    #Used for better code reproducability instead of initializing a constant value
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### 3.2 Move the model to GPU for better performance

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) 
print(device)

cuda


### 3.3 Define the optimizer and scheduler
- *We prefer AdamW optimizer function since it is the most widely used general purpose optimizer*
- *The scheduler will help bring the learning rate down if it senses overfitting*

In [19]:
epochs = 3
lr = 4e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

### 3.4 Define evaluation function with performance metrics

https://huggingface.co/transformers/main_classes/output.html

In [22]:
def model_eval(validation_dataloader):
    #---set the model in evaluation mode to disable dropout layers, batch normalization, etc
    model.eval()
    
    #---Initialize some variables to calculate model performance post evaluation
    total_loss = 0                          #counter to keep a sum of loss values across all batches
    predicted_class, true_class = [], []    #counters to keep track of all the predicted and true classes across all batches
    
    #---Use model to predict with validation dataset
    for batch in tqdm(validation_dataloader):
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                                                         #move the batch elements to GPU for faster performance
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}
        
        #---enable no_grad option in pytorch as there will be no back-propogation during evaluation
        with torch.no_grad():
            #---Predict outputs using the model
            model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]             #Example output in every iteration: tensor(0.2658, device='cuda:0')
        total_loss += loss.item()
        #print('model op 0:', loss)
        
        #---get the predicted outputs (logits) for current batch and append to predicted_class list
        logits = model_output[1]                          #List of lists [[0.1 0.2 0.3][0.1 0.2 0.3]...32 lists inside a list]
        #move the logits to cpu and convert to numpy since we will compute the simple math error calculations in cpu and not GPU
        logits = logits.detach().cpu().numpy()            #array of list of lists array([[0.1 0.2 0.3][0.1 0.2 0.3]...32 lists inside a list])
        predicted_class.append(logits)                    #list containing array of list of lists [array, array, ...]
        #print('model op 1:', logits)
        #print('pred_class:', predicted_class)
        
        #---get the original true labels and append to true_class list
        true_labels = input_dict['labels'].cpu().numpy()   #array of list of labels [0,1,2,2,1,0,...32 labels]
        true_class.append(true_labels)                     #list of array of list of labels [array([32 labels]), array([32 labels]),...]
        #print('true_class:', true_class)
    
    #---Compute average loss
    average_loss = total_loss/len(validation_dataloader) 
    
    #---Compute prediction accuracy
    predictions = np.concatenate(predicted_class, axis=0)  #list of lists [[0.1 0.2 0.3][0.1 0.2 0.3]....]
    true_vals = np.concatenate(true_class, axis=0)         #List of lists
    correct_predictions = np.sum(np.argmax(predictions, axis=1).flatten() == true_vals)
    prediction_accuracy = correct_predictions / len(training_data[training_data.dataset=='val'])
    #print('predicaitons', predictions)
    #print('true_vals', true_vals)
    
    #---Compute f1 score
    flattened_predictions = np.argmax(predictions, axis=1).flatten()    #List of values in predictions
    flattened_labels = true_vals.flatten()                              #List of values in true_vals
    f1score = f1_score(flattened_labels, flattened_predictions, average='weighted')
    #print('preds_flat:', preds_flat)
    #print('labels_flat:', labels_flat)
    
    return average_loss, predictions, true_vals, prediction_accuracy, f1score

### 3.5 Clear GPU cache

In [9]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=True)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| GPU reserved memory   |       0 B  |       0 B  |       0 B  |       0 B  |\n|--------------------------------------------------

- https://machinelearningmastery.com/exploding-gradients-in-neural-networks/
- https://neptune.ai/blog/understanding-gradient-clipping-and-how-it-can-fix-exploding-gradients-problem

## 4. Trial and error to find the best parameters
- *The original paper for BERT worked on batch sizes of (16 and 32) and learning rates of (5e-5, 4e-5, 3e-5, 2e-5, 1e-5)*
- REF: *https://arxiv.org/abs/1810.04805*
- *We prefer AdamW optimizer function since it is the most widely used general purpose optimizer and Adam was the optimization function used in the original paper on BERT*
- *The scheduler will help bring the learning rate down if it senses overfitting*

### 4.1 Learning Rate: 4e-5, Batch size = 32

In [24]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.4918999179587727
Validation loss: 0.41849138167424077
validation accuracy: 0.8373655913978495
f1 score: 0.8332252588220247




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.28421257756519736
Validation loss: 0.4393012648464268
validation accuracy: 0.8435483870967742
f1 score: 0.8401358283236026




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.14938459754785338
Validation loss: 0.5085086523770139
validation accuracy: 0.8448924731182795
f1 score: 0.8431335409156433



### 4.2 Learning Rate: 3e-5, Batch size = 32

In [25]:
epochs = 3
lr = 3e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

In [26]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.15330144212096494
Validation loss: 0.6249019023763318
validation accuracy: 0.8424731182795699
f1 score: 0.8359484382569047




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.08603929308697865
Validation loss: 0.7243434639376962
validation accuracy: 0.8365591397849462
f1 score: 0.832656661531438




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.048058861364509736
Validation loss: 0.7818541542873678
validation accuracy: 0.8405913978494624
f1 score: 0.8391119006742153



### 4.3 Learning Rate: 2e-5, Batch size = 32

In [27]:
epochs = 3
lr = 2e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

In [28]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.05204733631815979
Validation loss: 0.8366450298036265
validation accuracy: 0.8397849462365592
f1 score: 0.8379845839596098




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.031242405159374403
Validation loss: 0.9352583129309181
validation accuracy: 0.8381720430107527
f1 score: 0.8370533706716292




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.017575961076508124
Validation loss: 0.945866455630455
validation accuracy: 0.839247311827957
f1 score: 0.8368624594670694



### 4.4 Learning Rate: 1e-5, Batch size = 32

In [29]:
epochs = 3
lr = 1e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

In [30]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.020900169156075794
Validation loss: 1.0198117283164945
validation accuracy: 0.8400537634408602
f1 score: 0.8405335717969352




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.013037168813752942
Validation loss: 1.0609567559276445
validation accuracy: 0.8411290322580646
f1 score: 0.838543712798463




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.008980270750444158
Validation loss: 1.0620940329564703
validation accuracy: 0.839516129032258
f1 score: 0.8379400391411976



### 4.5 Learning Rate: 4e-5, Batch size = 16

In [31]:
batch_size = 16

# We Need two different dataloder
train_dloader = DataLoader(train_tensor, sampler=RandomSampler(train_tensor), batch_size=batch_size)
val_dloader = DataLoader(val_tensor, sampler=RandomSampler(val_tensor), batch_size=batch_size)

epochs = 3
lr = 4e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

In [32]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.11277784266268615
Validation loss: 1.0250013472502122
validation accuracy: 0.828763440860215
f1 score: 0.8245852248890445




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.06149198831927722
Validation loss: 1.1543163704504311
validation accuracy: 0.8354838709677419
f1 score: 0.8300636150507268




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.022426978693292396
Validation loss: 1.1787855487485224
validation accuracy: 0.8360215053763441
f1 score: 0.8348989907434257



#### 4.6 Learning Rate: 3e-5, Batch size = 16

In [33]:
batch_size = 16

# We Need two different dataloder
train_dloader = DataLoader(train_tensor, sampler=RandomSampler(train_tensor), batch_size=batch_size)
val_dloader = DataLoader(val_tensor, sampler=RandomSampler(val_tensor), batch_size=batch_size)

epochs = 3
lr = 3e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

In [34]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.05674324723129739
Validation loss: 1.216117960113807
validation accuracy: 0.828494623655914
f1 score: 0.8279334247486939




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.030379846902986833
Validation loss: 1.1918802885959463
validation accuracy: 0.8397849462365592
f1 score: 0.8348860859740733




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.016524545600257314
Validation loss: 1.2561901851937607
validation accuracy: 0.835752688172043
f1 score: 0.833709656659957



### 4.7 Learning Rate: 2e-5, Batch size = 16

In [35]:
batch_size = 16

# We Need two different dataloder
train_dloader = DataLoader(train_tensor, sampler=RandomSampler(train_tensor), batch_size=batch_size)
val_dloader = DataLoader(val_tensor, sampler=RandomSampler(val_tensor), batch_size=batch_size)

epochs = 3
lr = 2e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

In [36]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.02659671924670162
Validation loss: 1.3028597836966325
validation accuracy: 0.8344086021505376
f1 score: 0.8354077246675321




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.015641017653211818
Validation loss: 1.3521604862429888
validation accuracy: 0.8387096774193549
f1 score: 0.8365732260008416




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.009666627362538765
Validation loss: 1.3435795108581678
validation accuracy: 0.8405913978494624
f1 score: 0.8392133898282316



### 4.8 Learning Rate: 1e-5, Batch size = 16

In [37]:
batch_size = 16

# We Need two different dataloder
train_dloader = DataLoader(train_tensor, sampler=RandomSampler(train_tensor), batch_size=batch_size)
val_dloader = DataLoader(val_tensor, sampler=RandomSampler(val_tensor), batch_size=batch_size)

epochs = 3
lr = 1e-5
optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dloader)*epochs)

In [38]:
training_loss_dict = {}
validation_loss_dict = {}

for epoch in tqdm(range(1, epochs+1)):
    
    #---set the model to training mode
    model.train()         
    
    #counter to keep a sum of loss values across all epochs
    total_loss = 0

    # Setting up the Progress bar to Moniter the progress of training
    pbar = tqdm(train_dloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in pbar:
        
        #---set zero_grad so that gradient values are not accumulated across batches
        model.zero_grad()
        
        #---get the input_ids, attention_masks and labels from each batch int a dictionary to pass as input to the model
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
         
        input_dict = {'input_ids': temp_batch[0], 'attention_mask': temp_batch[1], 'labels': temp_batch[2]}       

        #---Predict outputs using the model
        model_output = model(**input_dict)

        #---get loss for current batch and add to the total_loss counter
        loss = model_output[0]
        total_loss += loss.item()
        
        #---compute and clip gradients
        loss.backward()                                           #compute gradients via backpropogation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #clip gradients to prevent gradient explosion issue

        #---Use optimizer and scheduler to tune the model during epochs
        optimizer.step()
        scheduler.step()
        
        pbar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #---save the model to disk        
    #torch.save(model.state_dict(), r'C:\Users\Karthik\Desktop\Dissertation\BERT\Model\BERT_trained.model')
        
    #---calculate metrics 
    loss_train_avg = total_loss/len(train_dloader)            
    val_loss, predictions, true_vals, val_accuracy, f1score = model_eval(val_dloader)

    training_loss_dict[epoch] = loss_train_avg
    validation_loss_dict[epoch] = val_loss
    
    tqdm.write("\u0332".join(f'\nEpoch {epoch}'))
    tqdm.write(f'Training loss: {loss_train_avg}')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'validation accuracy: {val_accuracy}')
    tqdm.write(f'f1 score: {f1score}')
    
torch.save(model, f"C:\\Users\\Karthik\\Desktop\\Dissertation\\BERT\\Models\\BERT_TwitterUSAirline_Uneven_Batch_{batch_size}_LR_{lr}")



̲E̲p̲o̲c̲h̲ ̲1
Training loss: 0.011044883322467012
Validation loss: 1.421291095974326
validation accuracy: 0.8400537634408602
f1 score: 0.8375344087357359




̲E̲p̲o̲c̲h̲ ̲2
Training loss: 0.010214573554559786
Validation loss: 1.4268254187153275
validation accuracy: 0.8379032258064516
f1 score: 0.8339374495444586




̲E̲p̲o̲c̲h̲ ̲3
Training loss: 0.006982891913814741
Validation loss: 1.4047865273845315
validation accuracy: 0.839247311827957
f1 score: 0.8381167321401644



## 4. Predict with the best model
- *Batch size of 32 and learning rate of 4e-4 gave the best results*

### 5.1 Load the selected model

In [10]:
reqd_model = torch.load(r"C:\Users\Karthik\Desktop\Dissertation\BERT\Models\BERT_TwitterUSAirline_Uneven_Batch_32_LR_4e-05")

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
reqd_model.to(device) 
print(device)

cuda


### 5.2 Prepare the datasets to be used in the BERT model for prediction

#### 5.2.1 Clean the text data

In [12]:
for tick, tick_df in reddit_df_dict.items():
#     print(tick, tick_df)
    tick_df['cleaned_body'] = tick_df['body'].apply(lambda x: text_preprocessing(x))
    tick_df['LEN'] = tick_df.cleaned_body.str.len()    

In [13]:
for tick, tick_df in twitter_df_dict.items():
    tick_df['cleaned_body'] = tick_df['cleaned_content'].apply(lambda x: text_preprocessing(x))
    tick_df['LEN'] = tick_df.cleaned_body.str.len()    

#### 5.2.2 Handle records where text is greater than a length of 512
- *BERT can only handle a max length of 512*
- *For each comment > length of 512, break the comment into multiple sets of 512*
- *After breaking, create a new record for each of the broken parts and append to the original dataframe. Since we take the average sentiment scores, appending new rows with the same dates will not affect the data*

In [14]:
n=512
def ffn(xseries):
    list_of_lists = []
        
    if xseries['LEN']>512:
        parts = [xseries['cleaned_body'][i:i+n] for i in range(0, xseries['LEN'], n)]
    
        counter=0
        for i in parts:
            list_series = []
            list_series.append(xseries['body'])
            list_series.append(xseries['created_utc'])
            list_series.append(xseries['id'])
            list_series.append(xseries['top'])
            list_series.append(xseries['year'])
            list_series.append(xseries['month'])
            list_series.append(xseries['day'])
            list_series.append(xseries['ticker'])
            list_series.append(xseries['YearMonDay'])
            list_series.append(i)
            counter+=1
            list_series.append(counter)
            
            list_of_lists.append(list_series)
    
        return list_of_lists
    else:
        return 0

In [15]:
for tick, tick_df in reddit_df_dict.items():
    tick_df['mltp'] = tick_df.apply(ffn, axis=1)

In [16]:
for tick, tick_df in twitter_df_dict.items():
    tick_df['mltp'] = tick_df.apply(ffn, axis=1)

In [17]:
for tick, tick_df in reddit_df_dict.items():
    addition_list = tick_df.query('mltp != 0')['mltp']
    
    if len(addition_list)==0:
        continue
    
    ind_additions = []

    for list_of_sentences in addition_list:
        for sentence_part in list_of_sentences:
            ind_additions.append(sentence_part)
    
    
    df_additions = pd.DataFrame(ind_additions, columns=['body','created_utc','id','top','year','month','day','ticker','YearMonDay','cleaned_body','LEN'])
    
    tick_df.drop('mltp', axis=1, inplace=True)
    tick_df.drop(tick_df[tick_df.LEN > 512].index, inplace = True)
    tick_df = tick_df.append(df_additions, ignore_index=True)

In [18]:
for tick, tick_df in twitter_df_dict.items():
    addition_list = tick_df.query('mltp != 0')['mltp']
    
    if len(addition_list)==0:
        continue
    
    ind_additions = []

    for list_of_sentences in addition_list:
        for sentence_part in list_of_sentences:
            ind_additions.append(sentence_part)
    
    
    df_additions = pd.DataFrame(ind_additions, columns=['body','created_utc','id','top','year','month','day','ticker','YearMonDay','cleaned_body','LEN'])
    
    tick_df.drop('mltp', axis=1, inplace=True)
    tick_df.drop(tick_df[tick_df.LEN > 512].index, inplace = True)
    tick_df = tick_df.append(df_additions, ignore_index=True)

- *Make sure there is no data where comment length is greater than 512*

In [19]:
for tick, tick_df in reddit_df_dict.items():
    print(tick_df.query('LEN > 512'))

Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []
Empty DataFrame
Columns: [body, created_utc, id, top, year, month, day, ticker, YearMonDay, cleaned_body, LEN]
Index: []


In [20]:
for tick, tick_df in twitter_df_dict.items():
    print(tick_df.query('LEN > 512'))

Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []
Empty DataFrame
Columns: [date, content, likeCount, lang, cleaned_content, year, month, day, ticker, YearMonDay, cleaned_body, LEN, mltp]
Index: []


#### 5.2.3 Define a funtion to prepare reddit and twitter data for model prediction
- *The data should be tokenized and encoded and then tensor datasets must be created from them to be passed to the BERT model*
- *We will create a dictionary for reddit and twitter to hold the tensor datasets respectively*

In [21]:
#---Initialize bert-base-uncased tokenizer and select the option to convert all text to lowercase
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def dataloader_fnc(text, MAX_LEN):
    input_id_list = []
    attn_mask_list = []
    
    for line in text:
        #--Tokenize and encode the cleaned text (we do not return_tensors here since our tensor should have all the lines)
        data_enc = tokenizer.encode_plus(line, 
                                         add_special_tokens=True, 
                                         return_attention_mask=True, 
                                         pad_to_max_length=True, 
                                         max_length=MAX_LEN
                                         )
        
        #--get the input_ids and attention_mask values
        input_id_list.append(data_enc.get("input_ids"))
        attn_mask_list.append(data_enc.get("attention_mask"))
    
    tensor_dataset = TensorDataset(torch.tensor(input_id_list), torch.tensor(attn_mask_list) )
    dataloader = DataLoader(tensor_dataset, sampler=SequentialSampler(tensor_dataset), batch_size=32)
    
    return dataloader

In [22]:
reddit_dataloader_dict = {tick: dataloader_fnc(tick_df.cleaned_body, tick_df.cleaned_body.str.len().max())  for tick, tick_df in reddit_df_dict.items()}

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2184: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
reddit_dataloader_dict

{'AAPL': <torch.utils.data.dataloader.DataLoader at 0x248ce4bbbb0>,
 'AMC': <torch.utils.data.dataloader.DataLoader at 0x248c6f48b50>,
 'DKNG': <torch.utils.data.dataloader.DataLoader at 0x248c6f489d0>,
 'TSLA': <torch.utils.data.dataloader.DataLoader at 0x248c6f48790>,
 'AMD': <torch.utils.data.dataloader.DataLoader at 0x248c6f48760>,
 'BABA': <torch.utils.data.dataloader.DataLoader at 0x248c6f48490>}

In [24]:
twitter_dataloader_dict = {tick: dataloader_fnc(tick_df.cleaned_body, tick_df.cleaned_body.str.len().max())  for tick, tick_df in twitter_df_dict.items()}

In [25]:
twitter_dataloader_dict

{'AAPL': <torch.utils.data.dataloader.DataLoader at 0x249a610e880>,
 'AMC': <torch.utils.data.dataloader.DataLoader at 0x249a610e430>,
 'DKNG': <torch.utils.data.dataloader.DataLoader at 0x249a610eac0>,
 'TSLA': <torch.utils.data.dataloader.DataLoader at 0x249a610ee80>,
 'AMD': <torch.utils.data.dataloader.DataLoader at 0x249a610ef10>,
 'BABA': <torch.utils.data.dataloader.DataLoader at 0x249a610ebe0>}

## 6. Prediction

### 6.1 Define a prediction function

In [26]:
def prediction(bert_model, data):
    #--put the model in eval mode
    bert_model.eval()
    
    #--predict and collect output logits in a list
    output_logit_list = []
    
    for batch in data:
        temp_batch = tuple(b.to(device) for b in batch)                 #move the batch elements to GPU for faster performance
        input_id_tensor = temp_batch[0]
        attn_mask_tensor = temp_batch[1]
#         print(input_id_tensor)
#         print(attn_mask_tensor)
        
        with torch.no_grad():
            output_logits = bert_model(input_id_tensor, attn_mask_tensor)
            output_logit_list.append(output_logits[0])
    
    #--concatenate logits across batches and apply softmax
    complete_logits = torch.cat(output_logit_list, dim=0)
    softmax_probs = Func.softmax(complete_logits, dim=1).cpu().numpy() #convert to numpy
    
    return softmax_probs

### 6.2 Predict sentiments for reddit data

#### 6.2.1 Prediction for reddit comments
- *Use the model to get positive and negative probabilities for each reddit comment*
- *Group by date and compute average positive, negative and neutral scores for each day*

In [27]:
for tick, tick_df in reddit_df_dict.items():
    pred_df = pd.DataFrame(prediction(reqd_model, reddit_dataloader_dict[tick]), columns=['neutral', 'positive', 'negative'])
    reddit_df_dict[tick] = pd.concat([tick_df, pred_df], axis=1)

In [28]:
for tick, tick_df in reddit_df_dict.items():
    tick_grouped = tick_df.groupby(['ticker', 'YearMonDay'])[['neutral', 'positive', 'negative']].mean().reset_index()
    tick_grouped.to_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Final_dfs\\reddit_{tick}_BERT_TwitterAirline_Uneven.pkl")
    print(tick_grouped)

    ticker YearMonDay   neutral  positive  negative
0     AAPL    20Aug03  0.526287  0.027239  0.446474
1     AAPL    20Aug05  0.550275  0.097208  0.352517
2     AAPL    20Aug06  0.502716  0.030566  0.466717
3     AAPL    20Aug10  0.491159  0.174718  0.334123
4     AAPL    20Aug11  0.430312  0.008052  0.561636
..     ...        ...       ...       ...       ...
253   AAPL    21May24  0.308020  0.022472  0.669508
254   AAPL    21May25  0.559654  0.077792  0.362554
255   AAPL    21May26  0.402627  0.030910  0.566463
256   AAPL    21May27  0.382373  0.035044  0.582582
257   AAPL    21May28  0.451543  0.032395  0.516062

[258 rows x 5 columns]
    ticker YearMonDay   neutral  positive  negative
0      AMC    20Aug05  0.010696  0.360304  0.629000
1      AMC    20Aug13  0.729069  0.003219  0.267712
2      AMC    20Aug17  0.066926  0.001672  0.931402
3      AMC    20Aug19  0.926393  0.014905  0.058702
4      AMC    20Aug25  0.904103  0.087432  0.008466
..     ...        ...       ...       ..

#### 6.2.2 Prediction for twitter comments
- *Use the model to get positive and negative probabilities for each twitter comment*
- *Group by date and compute average positive, negative and neutral scores for each day*

In [29]:
for tick, tick_df in twitter_df_dict.items():
    pred_df = pd.DataFrame(prediction(reqd_model, twitter_dataloader_dict[tick]), columns=['neutral', 'positive', 'negative'])
    twitter_df_dict[tick] = pd.concat([tick_df, pred_df], axis=1)

In [30]:
for tick, tick_df in twitter_df_dict.items():
    tick_grouped = tick_df.groupby(['ticker', 'YearMonDay'])[['neutral', 'positive', 'negative']].mean().reset_index()
    tick_grouped.to_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Final_dfs\\twitter_{tick}_BERT_TwitterAirline_Uneven.pkl")
    print(tick_grouped)

    ticker YearMonDay   neutral  positive  negative
0     AAPL    20Aug01  0.479722  0.095784  0.424494
1     AAPL    20Aug02  0.584998  0.067164  0.347838
2     AAPL    20Aug03  0.478685  0.052511  0.468804
3     AAPL    20Aug04  0.531005  0.051978  0.417017
4     AAPL    20Aug05  0.519848  0.070183  0.409970
..     ...        ...       ...       ...       ...
482   AAPL    21Sep26  0.373830  0.070840  0.555331
483   AAPL    21Sep27  0.508977  0.039590  0.451433
484   AAPL    21Sep28  0.485269  0.048223  0.466508
485   AAPL    21Sep29  0.555761  0.057389  0.386850
486   AAPL    21Sep30  0.515702  0.034823  0.449475

[487 rows x 5 columns]
    ticker YearMonDay   neutral  positive  negative
0      AMC    20Aug01  0.452338  0.186543  0.361119
1      AMC    20Aug02  0.428220  0.061719  0.510061
2      AMC    20Aug03  0.712510  0.028898  0.258592
3      AMC    20Aug04  0.665846  0.036564  0.297590
4      AMC    20Aug05  0.537198  0.106366  0.356436
..     ...        ...       ...       ..

#### 6.2.3 Calculate combined predictions by combining both reddit and twitter sentiments

In [31]:
for ticker in tick_list:
    print(ticker)
    reddit_sentis = reddit_df_dict[ticker][['ticker', 'YearMonDay', 'neutral', 'positive', 'negative']]
    twitter_sentis = twitter_df_dict[ticker][['ticker', 'YearMonDay', 'neutral', 'positive', 'negative']]
    
#     print(reddit_sentis,twitter_sentis,pd.concat([reddit_sentis,twitter_sentis], axis=0,ignore_index=True))
    combined_sentis = pd.concat([reddit_sentis,twitter_sentis], axis=0,ignore_index=True)
    combined_sentis_grouped = combined_sentis.groupby(['ticker', 'YearMonDay'])[['neutral', 'positive', 'negative']].mean().reset_index()
    combined_sentis_grouped.to_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Final_dfs\\combined_{ticker}_BERT_TwitterAirline_Uneven.pkl")
    print(combined_sentis_grouped)

AAPL
    ticker YearMonDay   neutral  positive  negative
0     AAPL    20Aug01  0.479722  0.095784  0.424494
1     AAPL    20Aug02  0.584998  0.067164  0.347838
2     AAPL    20Aug03  0.484684  0.049326  0.465990
3     AAPL    20Aug04  0.531005  0.051978  0.417017
4     AAPL    20Aug05  0.521134  0.071325  0.407542
..     ...        ...       ...       ...       ...
499   AAPL    21Sep26  0.373830  0.070840  0.555331
500   AAPL    21Sep27  0.508977  0.039590  0.451433
501   AAPL    21Sep28  0.485269  0.048223  0.466508
502   AAPL    21Sep29  0.555761  0.057389  0.386850
503   AAPL    21Sep30  0.515702  0.034823  0.449475

[504 rows x 5 columns]
AMC
    ticker YearMonDay   neutral  positive  negative
0      AMC    20Aug01  0.452338  0.186543  0.361119
1      AMC    20Aug02  0.428220  0.061719  0.510061
2      AMC    20Aug03  0.712510  0.028898  0.258592
3      AMC    20Aug04  0.665846  0.036564  0.297590
4      AMC    20Aug05  0.489334  0.129452  0.381214
..     ...        ...       ...

## References
- https://machinelearningmastery.com/exploding-gradients-in-neural-networks/
- https://neptune.ai/blog/understanding-gradient-clipping-and-how-it-can-fix-exploding-gradients-problem